In [12]:
import pandas as pd
import os
import yfinance as yf

In [13]:
def analyze_spike_or_drop(data):
    # Calculate daily returns
    data = data.copy() 
    
    data['Daily Return'] = data['Close'].pct_change()

    # Find dates where the stock spiked up by 50% or more or dropped by 50% or more
    significant_dates = data[data['Daily Return'].abs() >= 0.15].index

    days_after_list = [1, 2, 3, 4, 5, 6, 7, 14, 28]
    results_df_list = []

    for significant_date in significant_dates:
        results = {'day0': data.loc[significant_date, 'Daily Return']}
        for days_after in days_after_list:
            try:
                price_before = data.loc[significant_date, 'Adj Close']
                price_after = data.loc[data.index[data.index.get_loc(significant_date) + days_after], 'Adj Close']
                price_change = (price_after - price_before) / price_before
                results[f"{days_after} days after"] = price_change
            except:
                results[f"{days_after} days after"] = None

        results_df = pd.DataFrame(results, index=[significant_date])
        results_df_list.append(results_df)

    if not results_df_list:
        print("No significant movements detected.")
        return None

    final_df = pd.concat(results_df_list)

    def color_cells(val):
        if val is None:
            return 'background-color: #D3D3D3'  # Gray for None values
        elif val >= 0:
            return 'background-color: green'
        else:
            return 'background-color: red'

    styled_df = final_df.style.applymap(color_cells).format("{:.2%}")

    # Calculate the average percentage change for each time frame
    avg_changes = final_df.mean()

    # Calculate the frequency of positive returns for each time frame
    positive_freq = (final_df > 0).sum() / final_df.count()

    # Calculate the frequency of negative returns for each time frame
    negative_freq = (final_df < 0).sum() / final_df.count()

    summary_df = pd.DataFrame({
        'Average Change': avg_changes,
        'Frequency of Positive Returns': positive_freq,
        'Frequency of Negative Returns': negative_freq
    })

    # Formatting the summary DataFrame
    summary_styled = summary_df.style.format({
        'Average Change': "{:.2%}",
        'Frequency of Positive Returns': "{:.2%}",
        'Frequency of Negative Returns': "{:.2%}"
    })
    
    # Build the results list
    results_list = []
    for index, row in final_df.iterrows():
        # Determine the direction based on the 'Daily Return' of 'day0'
        direction = 'up' if data.loc[index, 'Daily Return'] >= 0 else 'down'
        
        for col in final_df.columns:
            days_after = 0 if col == 'day0' else int(col.split()[0])
            return_value = row[col]
            results_list.append({
                'Direction': direction,
                'Days After': days_after,
                'Return': return_value
            })
            
    return styled_df, summary_styled, results_list



In [14]:
tickers = ['BTC-USD']


all_results = []

for ticker in tickers:
    try:
        # Fetch data from yfinance for the past 10 years (modify the period if needed)
        data = yf.download(ticker, period="10y")[['Close', 'Adj Close']]
        print(f"Analyzing {ticker}...")
        result = analyze_spike_or_drop(data)
        if result:
            display_df, summary_styled, results_for_ticker = result
            all_results.extend(results_for_ticker)
            display(display_df)

    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")

[*********************100%%**********************]  1 of 1 completed
Analyzing BTC-USD...


,day0,1 days after,2 days after,3 days after,4 days after,5 days after,6 days after,7 days after,14 days after,28 days after
2014-11-12 00:00:00,15.19%,-0.67%,-6.08%,-11.20%,-8.42%,-8.54%,-11.42%,-10.15%,-13.03%,-18.23%
2015-01-13 00:00:00,-15.66%,-21.14%,-7.09%,-7.87%,-11.78%,-6.87%,-4.87%,-6.44%,16.65%,-2.67%
2015-01-14 00:00:00,-21.14%,17.82%,16.84%,11.88%,18.10%,20.64%,18.65%,27.40%,31.34%,23.07%
2015-01-15 00:00:00,17.82%,-0.83%,-5.04%,0.24%,2.39%,0.70%,8.13%,11.23%,11.28%,5.68%
2015-08-18 00:00:00,-18.18%,7.39%,11.50%,10.18%,9.15%,8.10%,-0.28%,4.99%,8.07%,9.11%
2016-01-15 00:00:00,-15.33%,6.37%,4.93%,6.27%,4.34%,15.34%,12.61%,4.98%,4.16%,5.47%
2017-07-17 00:00:00,15.47%,4.06%,2.02%,26.44%,19.72%,26.10%,22.53%,23.62%,29.03%,94.09%
2017-07-20 00:00:00,23.94%,-5.32%,-0.27%,-3.09%,-2.23%,-8.56%,-10.23%,-5.18%,-0.46%,53.74%
2017-09-14 00:00:00,-18.74%,15.30%,14.90%,13.56%,28.85%,24.41%,23.80%,15.09%,32.32%,72.65%
2017-09-15 00:00:00,15.30%,-0.34%,-1.50%,11.76%,7.90%,7.38%,-0.18%,-0.19%,14.45%,55.25%


In [15]:
df = pd.DataFrame(all_results, columns=['Direction', 'Days After', 'Return'])

summary = {
    'average stocks up': df[df['Direction'] == 'up'].groupby('Days After')['Return'].mean(),
    'average stocks down': df[df['Direction'] == 'down'].groupby('Days After')['Return'].mean(),
    'average stocks up_median': df[df['Direction'] == 'up'].groupby('Days After')['Return'].median(),
    'average stocks down_median': df[df['Direction'] == 'down'].groupby('Days After')['Return'].median(),
    'average stocks up_std': df[df['Direction'] == 'up'].groupby('Days After')['Return'].std(),
    'average stocks down_std': df[df['Direction'] == 'down'].groupby('Days After')['Return'].std(),
    'total stocks up': df[df['Direction'] == 'up'].groupby('Days After')['Return'].count(),
    'total stocks down': df[df['Direction'] == 'down'].groupby('Days After')['Return'].count()
}

pd.DataFrame(summary).T

Days After,0,1,2,3,4,5,6,7,14,28
average stocks up,0.184329,0.021916,-0.001666,0.022415,0.037395,0.051340,0.045770,0.047771,0.074461,0.187262
average stocks down,-0.194471,0.050302,0.061687,0.058889,0.087172,0.084695,0.065243,0.094561,0.188020,0.205066
average stocks up_median,0.178217,0.001225,-0.002655,0.032133,0.036362,0.062931,0.062222,0.066617,0.097305,0.096488
average stocks down_median,-0.168548,0.073930,0.049318,0.084801,0.091487,0.080965,0.053845,0.049887,0.166536,0.091080
average stocks up_std,0.034338,0.085755,0.081730,0.111685,0.076648,0.105155,0.115192,0.100839,0.126249,0.349887
average stocks down_std,0.069032,0.119833,0.077715,0.095743,0.141643,0.139808,0.118632,0.148770,0.234871,0.344814
total stocks up,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
total stocks down,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [16]:
btc_up = df[(df['Direction'] == 'up') & (df['Return']>0)].groupby('Days After').count()[['Direction']] / df[df['Direction'] == 'up'].groupby('Days After').count()[['Direction']]
btc_down = df[(df['Direction'] == 'down') & (df['Return']>0)].groupby('Days After').count()[['Direction']] / df[(df['Direction'] == 'down')].groupby('Days After').count()[['Direction']]



In [17]:
btc_up

,Direction
Days After,
0,1.000000
1,0.545455
2,0.363636
3,0.636364
4,0.727273
5,0.727273
6,0.636364
7,0.636364
14,0.727273


In [11]:
btc_down

,Direction
Days After,
0,NaN
1,0.666667
2,0.777778
3,0.777778
4,0.777778
5,0.777778
6,0.555556
7,0.666667
14,0.777778


In [18]:
data = yf.download(ticker, period="10y")[['Close', 'Adj Close']]

[*********************100%%**********************]  1 of 1 completed


In [20]:
data.to_csv('btc_data.csv')